# LQB385 Bioinformatics Workshop 3 - Analysis Notebook
(Write scenario here - I suggest that this notebook was provided to the student by their bioinformatician; they now need to modify the code in here to answer questions from the PI   *-Mitchell*)

In [ ]:
# Project startup (always upload analysed_GSE63310.RData and run this cell when notebook is first opened)
# NOTE TO USER ---------------------------
# Do not modify code in this cell!
# Else it won't work!

# Install packages
install.packages(
    packages = c(
        "ggpubr",
        "ggsignif",
        "limma",
        "magrittr",
        "metan",
        "rlang",
        "tibble",
        "tidyverse"
    ),
    repos = c(
        "https://repo.r-wasm.org",
        "https://bioc.r-universe.dev",
        "https://nepem-ufsc.r-universe.dev",
        "https://rstudio.r-universe.dev",
        "https://r-lib.r-universe.dev",
        "https://tidyverse.r-universe.dev"
    )
)

# Import packages
library("ggpubr")
library("ggsignif")
library("limma")
library("magrittr")
library("metan")
library("rlang")
library("tibble")
library("tidyverse")

# Define functions used in this notebook
symbol_to_entrez <- function(EList, symbols) {
    # This function accepts an EList containing voom-normalised logCPM data; a vector of characters representing genes of interest
    # and outputs the corresponding indices.
    # With error catching!
    indices <- which(EList$genes$SYMBOL %in% symbols)

    ## Check: Error if there are fewer ENTREZ IDs than gene symbols (impossible)
    if (length(indices) < length(symbols)) {
        stop(
            "There are fewer gene IDs than there should be. Check your gene symbols!"
        )
    }

    return(indices)
}

voom_to_long <- function(EList, symbols) {
    # This function accepts an EList containing voom-normalised logCPM data;
    # a character representing gene of interest
    # and outputs a dataframe ready for tidyverse data analysis

    # Get indices of selected genes
    indices <- symbol_to_entrez(EList, symbols)

    # Filter EList for selected indices
    # Rename rownames into gene symbols
    # Then transpose
    # Then convert into dataframe
    # Build 2 cases: if there is one gene, treat extracted data as a vector.
    # If there are multiple, treat extracted data like a df
    if (length(indices) == 1) {
        # Add column for sample_id
        lcpm_selected <- EList$E[indices, ]
        lcpm_selected <- tibble(
            sample_id = names(lcpm_selected),
            logCPM = lcpm_selected
        )

        # Get samplesheet
        # Append expression data
        # Make tidy
        lcpm_output <- EList$targets %>%
            rownames_to_column("sample_id") %>%
            left_join(
                lcpm_selected,
                by = join_by("sample_id" == "sample_id"),
                unmatched = "error"
            )

        # Return data
        return(lcpm_output)
    } else {
        # Add column for sample_id
        lcpm_selected <- EList$E[indices, ] %>%
            set_rownames(symbols) %>%
            t() %>%
            as.data.frame() %>%
            rownames_to_column("sample_id")

        # Get samplesheet
        # Append expression data
        # Make tidy
        lcpm_output <- EList$targets %>%
            rownames_to_column("sample_id") %>%
            left_join(
                lcpm_selected,
                by = join_by("sample_id" == "sample_id"),
                unmatched = "error"
            ) %>%
            pivot_longer(
                cols = symbols,
                names_to = "SYMBOL",
                values_to = "logCPM"
            )

        # Return data
        return(lcpm_output)
    }
}

get_signif_gse <- function(MArrayLM, symbol, contrasts) {
    # This function accepts a contrasted MArrayLM,
    # a character representing the gene of interest
    # a vector of contrasts to extract data from
    # and outputs a vector of adjusted p values.

    out <- map(contrasts, \(contrast) {
        topTable(
            fit = MArrayLM,
            coef = contrast,
            number = Inf,
            sort.by = "none"
        ) %>%
            filter(SYMBOL == symbol) %>%
            .$adj.P.Val
    }) %>%
        unlist() %>%
        stars_pval()

    return(out)
}

# Load data
load("./analysed_GSE63310.Rdata")

# Show list of imported packages (debug only)
sessionInfo()
# Show list of data (debug only)
# ls()

# Analysis of genes of interest

This section includes code cells for the following analyses:
- Gene expression histogram
  - Visualisation of expression of one gene.
- Gene expression boxplots
  - Visualisation of expression of one gene across experimental conditions.
- Correlation scatterplot
  - Visualisation of correlation between two selected genes.
- Correlation heatmap
  - Visualisation of correlation between selected genes.

All these analyses use logCPM data.

## Gene expression histogram
The following code produces a histogram of gene expression for the selected gene(s).

Use `?ggplot` and `?geom_histogram` to understand the code further.

In [ ]:
# Set gene(s) to be plotted
## Remember that Mus Musculus gene names are in Sentence Case
## e.g. Sulf1 Acta1
current_gene <- "Sulf1"

# Extract gene expression data
lcpm_histogram <- voom_to_long(GSE_logCPM, current_gene)

# Construct histogram
plot_histogram <- ggplot(
    lcpm_histogram,
    aes(x = logCPM)
) +
    geom_histogram() +
    theme_minimal() +
    ggtitle(
      label = current_gene |>
        str_replace_all("_", " ") |>
        str_wrap(width = 20, whitespace_only = TRUE)
    )

# Draw histogram
plot(plot_histogram)

## Gene expression boxplots
The following code produces a boxplot of gene expression changes for the selected gene.

Statistical analysis was performed using the limma moderated t-test. Adjusted p values: `<0.05: *. <0.01: **. <0.001: ***. <0.0001: ****`

Use `?ggplot` and `?geom_boxplot` to understand the code further.

In [ ]:
# Set gene(s) to be plotted
current_gene <- "Sulf1"
if (length(current_gene) > 1) stop("This plot can only handle 1 gene")

# Extract gene expression data
lcpm_boxplot <- voom_to_long(GSE_logCPM, current_gene)

# Construct histogram
plot_box <- ggplot(
    lcpm_boxplot,
    aes(x = group,y = logCPM)
) +
    geom_boxplot(aes(colour = group), outliers = FALSE) +
    geom_jitter(
        size = 1.2,
        stroke = 0.5,
        alpha = 0.5
    ) + 
    geom_signif(
        comparisons = list(
            c("Basal", "LP"),
            c("Basal", "ML"),
            c("LP", "ML")
        ),
        annotation = get_signif_gse(
            GSE_fitted,
            current_gene,
            c("lp_vs_basal", "ml_vs_basal", "ml_vs_lp")
        ),
        y_position = rep(max(lcpm_boxplot$logCPM) * 1.1, 3),
        textsize = 3,
        step_increase = 0.15
    ) +
    scale_y_continuous(expand = expansion(0, 0.4)) +
    theme_minimal() +
    ggtitle(
      label = current_gene |>
        str_replace_all("_", " ") |>
        str_wrap(width = 20, whitespace_only = TRUE)
    )

# Draw histogram
plot(plot_box)


## Gene-gene correlation scatterplot
The following code produces a scatterplot of gene expression for the selected 2 genes.

Statistical analysis was performed using Pearson's correlation.

In [ ]:
# Set gene(s) to be plotted
current_gene <- c("Gpc1","Sulf1")

if (length(current_gene) != 2) stop("This plot can only handle 2 genes")

# Extract gene expression data
lcpm_scatter <- voom_to_long(GSE_logCPM, current_gene)
lcpm_scatter <- pivot_wider(lcpm_scatter, names_from = SYMBOL, values_from = logCPM)

# Construct plot
## Define x and y axis based on selected genes
scatter_aesthetics <- current_gene %>% 
    set_names(c("x", "y")) %>%
    map(sym) %>%
    as_quosures(env = .GlobalEnv)

## Draw plot with defined aesthetics
plot_scatter <- ggplot(
    lcpm_scatter,
    aes(!!!scatter_aesthetics)
) +
    geom_point(
        aes(color = group),
        size = 1.2,
        stroke = 0.5,
        alpha = 0.8
    ) +
    geom_smooth(
        colour = "grey80",
        method = "lm",
        se = FALSE
    ) +
    stat_cor(
        aes(label = paste(..r.label.., ..p.label.., sep = "~`,`~")),
        r.accuracy = 0.01,
        alpha = 1
    ) +
    theme_minimal()

# Draw plot
plot(plot_scatter)


## Gene-gene correlation heatmap
The following code produces a heatmap of gene expression correlation for the selected genes.

Statistical analysis was performed using Pearson's correlation.

In [9]:
# Set gene(s) to be plotted
current_gene <- c("Gpc1","Sulf1", "Clu", "Basp1", "Cd82")

if (length(current_gene) < 2) stop("This plot needs at least 2 genes")

# Extract gene expression data

# Construct plot

# Draw plot


# Advanced details

## R syntax - variable assignment

In some cases throughout this workshop, you will need to assign one or many genes to a variable.

To assign one gene, use: `x <- "Gene"`

In [ ]:
# Demonstration: assignment of one character to one variable
x <- "Gene"

# Show content of x
print(x)

# Clean up (remove x)
rm(x)

To assign multiple genes (a vector of gene symbols), use: `x <- c("Gene1", "Gene2", "Gene3", "Gene4")`

In [ ]:
# Demonstration: assignment of a vector to one variable
x <- c("Gene1", "Gene2", "Gene3", "Gene4")

# Longer vectors can span over multiple lines
x <- c(
    "Gene1",
    "Gene2",
    "Gene3",
    "Gene4"
)

# Show content of x
print(x)

# Clean up (remove x)
rm(x)

Common errors:
- **R is case-sensitive!** Make sure you use the correct case when referring to your genes.
- Characters must be wrapped in "quotation marks".

## R syntax - exploring objects in this project

The uploaded `analysed_GSE63310.Rdata` includes the following objects:
- `GSE_logCPM`: Log-counts per million RNA-seq data (with accompanying gene and sample information)
- `GSE_fitted`: Test results of RNA-seq data (with accompanying gene and sample information)
- `GSE_camera`: Gene set analysis results of RNA-seq data
- `GSE_pca`: Principal Component Analysis results of RNA-seq data

Below are code examples for accessing these data:

- Accessing log-counts per million data

In [ ]:
GSE_logCPM[["E"]]

- Accessing gene information

In [ ]:
GSE_logCPM[["genes"]]

- Accessing sample information

In [ ]:
GSE_logCPM[["targets"]]

- Accessing comparisons definition (in this context, comparisons are called "contrasts")

In [ ]:
# Accessing contrasts
print(GSE_fitted[["contrasts"]])

# Accessing names of contrasts
print(colnames(GSE_fitted[["contrasts"]]))

- Accessing gene set analysis results

In [ ]:
# Note: use GSE_camera[["name of contrasts"]]
GSE_camera[["ml_vs_lp"]]